In [1]:
import pyspark.conf
import pyspark.sql
SparkConf = pyspark.conf.SparkConf
SparkSession = pyspark.sql.SparkSession
spark = SparkSession.builder \
            .appName("Intro") \
            .config('spark.executor.memory', '2g') \
            .config('spark.driver.memory','8g') \
            .getOrCreate()

In [113]:
from pyspark.sql import functions as F

In [ ]:
from utils import *

In [4]:
preview = spark.read.csv("data/linkage/")

In [5]:
preview.show()

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
|  _c0|  _c1|         _c2|         _c3|         _c4|         _c5|    _c6|   _c7|   _c8|   _c9|   _c10|    _c11|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
| 3148| 8326|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|14055|94934|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|33948|34740|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|  946|71870|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|    TRUE|
|64880|71676|           1|           ?|           1|           ?|      1|     1|     1|     1|      1|  

In [8]:
preview.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: string (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: string (nullable = true)
 |-- _c5: string (nullable = true)
 |-- _c6: string (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: string (nullable = true)
 |-- _c10: string (nullable = true)
 |-- _c11: string (nullable = true)



In [9]:
parsed = spark.read \
            .option("header", "true") \
            .option("nullValue", "?") \
            .option("inferSchema", "true") \
            .csv("data/linkage/")

In [34]:
parsed.show()

+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| id_1| id_2|cmp_fname_c1|cmp_fname_c2|cmp_lname_c1|cmp_lname_c2|cmp_sex|cmp_bd|cmp_bm|cmp_by|cmp_plz|is_match|
+-----+-----+------------+------------+------------+------------+-------+------+------+------+-------+--------+
| 3148| 8326|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|14055|94934|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|33948|34740|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|  946|71870|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|64880|71676|           1|        null|           1|        null|      1|     1|     1|     1|      1|    true|
|25739|45991|           1|        null|           1|        null|      1|     1|     1|     1|      1|  

In [35]:
parsed.printSchema()

root
 |-- id_1: string (nullable = true)
 |-- id_2: string (nullable = true)
 |-- cmp_fname_c1: string (nullable = true)
 |-- cmp_fname_c2: string (nullable = true)
 |-- cmp_lname_c1: string (nullable = true)
 |-- cmp_lname_c2: string (nullable = true)
 |-- cmp_sex: string (nullable = true)
 |-- cmp_bd: string (nullable = true)
 |-- cmp_bm: string (nullable = true)
 |-- cmp_by: integer (nullable = true)
 |-- cmp_plz: integer (nullable = true)
 |-- is_match: boolean (nullable = true)



In [36]:
parsed.count()

5749133

In [37]:
parsed.cache()

DataFrame[id_1: string, id_2: string, cmp_fname_c1: string, cmp_fname_c2: string, cmp_lname_c1: string, cmp_lname_c2: string, cmp_sex: string, cmp_bd: string, cmp_bm: string, cmp_by: int, cmp_plz: int, is_match: boolean]

In [38]:
parsed.groupBy("is_match").count().orderBy("count", ascending=False).show()

+--------+-------+
|is_match|  count|
+--------+-------+
|   false|5728201|
|    true|  20931|
|    null|      1|
+--------+-------+



In [39]:
parsed.createOrReplaceTempView("linkage")

In [40]:
spark.sql("""
            SELECT is_match, COUNT(*) cnt
            FROM linkage
            GROUP BY is_match
            ORDER BY cnt DESC
        """).show()

+--------+-------+
|is_match|    cnt|
+--------+-------+
|   false|5728201|
|    true|  20931|
|    null|      1|
+--------+-------+



In [41]:
summary = parsed.describe()
summary.show()

+-------+--------------------+-------------------+--------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+
|summary|                id_1|               id_2|        cmp_fname_c1|      cmp_fname_c2|      cmp_lname_c1|      cmp_lname_c2|            cmp_sex|             cmp_bd|            cmp_bm|            cmp_by|            cmp_plz|
+-------+--------------------+-------------------+--------------------+------------------+------------------+------------------+-------------------+-------------------+------------------+------------------+-------------------+
|  count|             5749133|            5749133|             5748126|            103699|           5749133|              2465|            5749133|            5748338|           5748338|           5748337|            5736289|
|   mean|   33324.47979999771|  66587.42400114964|   0.712902346423786|0.9000089989364373|0.

In [42]:
summary.select("summary", "cmp_fname_c1", "cmp_fname_c2").show()

+-------+--------------------+------------------+
|summary|        cmp_fname_c1|      cmp_fname_c2|
+-------+--------------------+------------------+
|  count|             5748126|            103699|
|   mean|   0.712902346423786|0.9000089989364373|
| stddev|  0.3887584395082905|0.2713306768152375|
|    min|                   0|                 0|
|    max|2.68694413843136e-05|                 1|
+-------+--------------------+------------------+



In [43]:
matches = parsed.where("is_match = true")
misses  = parsed.where("is_match = false")

In [44]:
matchSummary = matches.describe()
missSummary = misses.describe()

In [83]:
len(a)

12

<function pyspark.sql.functions.first(col, ignorenulls=False)>

In [114]:
def longForm(desc):
    schema = desc.schema
    
    def _tmp(row):
        metric = str(row[0])
        output = []
        for i in range(len(row)):
            output.append([metric, schema[i].name, str(row[i])])
        # end for
        return output
    # end def
    
    return desc.rdd.flatMap(lambda row : _tmp(row)).toDF(["metric", "field", "value"])
# end def
        
def pivotSummary(desc):
    lf = longForm(desc)
    return lf.groupBy("field"). \
        pivot("metric", ("count", "mean", "stddev", "min", "max")). \
        agg(F.first("value"))
# end def
        
def crossTabs(scored, t):
    return scored. \
      selectExpr("score >= %s as above" % t, "is_match").\
      groupBy("above").\
      pivot("is_match", ("true", "false")).\
      count()
# end def

def scoreMatchData(md):
    return md.cmp_lname_c1 + md.cmp_plz + \
            md.cmp_by + md.cmp_bd + md.cmp_bm
# end def

In [115]:
matchSummaryT = pivotSummary(matchSummary)
missSummaryT = pivotSummary(missSummary)
matchSummaryT.createOrReplaceTempView("match_desc")
missSummaryT.createOrReplaceTempView("miss_desc")
spark.sql("""
    SELECT a.field, a.count + b.count total, a.mean - b.mean delta
    FROM match_desc a INNER JOIN miss_desc b ON a.field = b.field
    ORDER BY delta DESC, total DESC
""").show()

+------------+---------+--------------------+
|       field|    total|               delta|
+------------+---------+--------------------+
|        id_1|5749132.0|  1255.8076310367542|
|     cmp_plz|5736289.0|  0.9563812499852176|
|cmp_lname_c2|   2464.0|  0.8064147192926263|
|      cmp_by|5748337.0|  0.7762059675300512|
|      cmp_bd|5748337.0|   0.775442311783404|
|cmp_lname_c1|5749132.0|  0.6838772482598161|
|      cmp_bm|5748337.0|  0.5109496938298685|
|cmp_fname_c1|5748125.0| 0.28545290574713356|
|cmp_fname_c2| 103698.0| 0.09104268062278553|
|     cmp_sex|5749132.0|0.032408185250332844|
|        id_2|5749132.0| -15383.483201807663|
|     summary|     null|                null|
+------------+---------+--------------------+



In [11]:
matchData = parsed.rdd.map(MatchData)

In [32]:
scored = matchData.map(lambda md: (str(scoreMatchData(md)), str(md.is_match).lower())).toDF(["score", "is_match"])

In [33]:
crossTabs(scored, 4.0).show()

+-----+-----+-------+
|above| true|  false|
+-----+-----+-------+
| true|20871|    637|
|false|   60|5727564|
+-----+-----+-------+

